In [2]:
import pandas as pd
import glob

In [74]:
# 전체 알고리즘
# 1. yolo 예측에서 class agnostic nms 해서 중복된 박스 제거
# 2. 박스대로 패치 뜯어내기
# 3. 분류 모델로 예측해 logit 계산
# 4. logit을 softmax 확률로 변환 후, 원본 이미지명 별로 확률, x, y, w, h 저장한 dict 생성
    # file_name: [x1, y1, w, h, prob]
    # prob = [class 0, class 1, ..., class34]
# 5. yoloy 예측을 원본 이미지명 별로 클래스, 확률, x, y, w, h 저장한 dict 생성
    # file_name: [class_id, conf, x1, y1, w, h]
# 6. test 이미지명 전체를 돌면서 yolo dict의 이미지 별 Bbox들을 돌면서 분류 모델의 bbox들과 iou가 0.5 이상인 bbox의 확률을 분류 모델의 bbox 의 확률로 보정
# 보정 방법
    # 6-1. 패치별로 yolo 예측은 여러 개, 분류 모델 확률은 한 개 존재
    # 6-2. yoloy bbox들의 conf를 클래스 위치에 맞게 분류 모델 확률의 자리에 더하기
    # 6-2. 더해진 확률을 softmax로 변환
    # 6-3. softmax 확률중 top 3의 클래스와 확률을 저장
    # 6-4. top 3의 클래스와 확률을 가장 conf가 높았던 yolo의 Bbox 좌표에 저장(즉, 총 3개의 bbox 저장됨)
# 7. 6.의 결과를 df 변환 후 저장

In [8]:
# 1. yolo 예측에서 class agnostic nms 해서 중복된 박스 제거
# 2. 박스대로 패치 뜯어내기
from collections import defaultdict
from tqdm import tqdm

csv_dir = './results/0615_nms.csv'
yolo_csv = pd.read_csv(csv_dir)

def csv2dict(df):
    image_dict = defaultdict(list)
    for i in range(len(df)):
        file_name, class_id, confidence, point1_x, point1_y, point2_x, point2_y, point3_x, point3_y, point4_x, point4_y = df.iloc[i]
        image_dict[file_name].append([class_id, confidence, point1_x, point1_y, point2_x, point2_y, point3_x, point3_y, point4_x, point4_y])
    
    for k,v in image_dict.items():
        image_dict[k] = sorted(v, key=lambda x: x[1], reverse=True)

    return image_dict

image_dict = csv2dict(yolo_csv)
len(image_dict.keys())


3400

In [10]:
# 2-1. 박스대로 패치 뜯어내기
import cv2
base_dir = './open/test/'
crop_images = defaultdict(list)


for image_name, infos in tqdm(image_dict.items(), total=len(image_dict.keys())):
    image_path = base_dir + image_name
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

    for i, info in enumerate(infos):
        class_id, confidence, point1_x, point1_y, point2_x, point2_y, point3_x, point3_y, point4_x, point4_y = info
        xmin, ymin, xmax, ymax = point1_x, point1_y, point3_x, point3_y

        xmin, ymin, xmax, ymax = point1_x, point1_y, point3_x, point3_y
        w = xmax - xmin
        h = ymax - ymin
        
        crop_images[image_name].append([f'{image_name.replace(".png","")}_{i}_{class_id}_{xmin}_{ymin}_{w}_{h}.png', image[int(point1_y):int(point3_y), int(point1_x):int(point3_x)], class_id])
 

100%|██████████| 3400/3400 [02:44<00:00, 20.71it/s]


In [11]:
# 2-2. 패치 저장
import os

# 저장할 폴더 생성
csv_name = csv_dir.split('/')[-1].split('.')[0]
save_dir = f'./open/{csv_name}_images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 저장
for image_name, infos in tqdm(crop_images.items(), total=len(crop_images.keys())):
    for info in infos:
        name, image, label = info
        image_save_dir = os.path.join(save_dir, name)
        cv2.imwrite(image_save_dir, image)

100%|██████████| 3400/3400 [00:56<00:00, 59.97it/s] 


In [13]:
# 2-3. 분류 모델 예측으로 줄 df 생성: 여기까지 실행하고 분류 모델로 추론
cnn_pred = pd.read_csv('./fold1234_ft_nms_images_result.csv')
cnn_pred.head()

,crop_name,class_id,x1,y1,w,h,0,1,2,3,...,25,26,27,28,29,30,31,32,33,34
0,064442001_0_16_1138_181_1429_496.png,16,1138,181,1429,496.png,0.827098,-1.205847,3.954166,2.012789,...,-1.733715,-1.387932,0.911139,-2.202701,-1.902774,-1.421460,-0.211629,1.386167,-0.515668,-1.152198
1,064507368_0_16_492_124_761_429.png,16,492,124,761,429.png,2.671357,0.625286,4.496533,5.001987,...,-0.891321,-0.917498,1.391235,-2.909070,-0.590935,-0.714352,-0.819183,-2.095201,2.473240,-0.093439
2,065131036_0_3_1108_48_1359_314.png,3,1108,48,1359,314.png,-0.140470,0.942809,1.103673,11.764345,...,0.690366,-0.549596,1.401362,-3.682058,2.037853,-1.303285,0.239764,7.122533,3.636045,-3.969671
3,065131036_1_25_837_440_1171_853.png,25,837,440,1171,853.png,-3.162100,-1.767774,-0.238539,0.920422,...,0.680742,-0.416739,8.421043,-0.054986,-0.454342,0.465129,-1.836299,-0.975888,1.263845,7.064037
4,065147868_0_16_727_188_1010_500.png,16,727,188,1010,500.png,1.584079,-1.001038,7.297412,5.495926,...,-0.628504,-1.166090,1.521971,-4.917948,-2.728323,-1.718698,-1.005397,1.337893,1.893246,-1.378787


In [27]:
############ 분류 모델 추론 후 여기부터 실행 ############
import pandas as pd
from collections import defaultdict
import numpy as np

yolo_pred = pd.read_csv('./results/wbf_0616_conf.csv') # yolo 예측

cnn_pred = pd.read_csv('./open/0615_nms_images.csv') # 분류 모델 예측
cnn_pred

,crop_name,0,1,2,3,4,5,6,7,8,...,25,26,27,28,29,30,31,32,33,34
0,111212079_0_33_1120_114_249_254.png,1.325544,1.529212,-0.842277,4.613559,-2.161958,-2.435503,0.426466,1.763330,-3.885303,...,-2.148974,-4.357024,3.610795,-0.719268,-1.847862,-0.136731,7.676729,6.501480,6.327097,-0.497033
1,132929363_0_13_1230_305_277_258.png,-1.460786,0.227570,0.312908,3.285975,-4.928470,3.175369,-0.973748,7.062544,-5.348039,...,-4.652607,-5.570615,3.789585,1.063114,-3.933642,-3.523746,2.185755,3.462498,1.305163,5.276254
2,110903616_0_6_1216_365_289_267.png,-4.003664,-3.454632,-0.693605,-2.037188,-3.185054,1.096819,8.486217,9.061948,-0.428196,...,-1.733481,0.099227,0.231578,3.219274,-0.577177,0.387547,1.059032,-1.956483,-0.396154,0.632401
3,104445026_2_11_0_0_186_101.png,-0.553749,-0.543294,0.752686,0.286904,-0.866220,-0.607441,-0.463283,1.307225,-1.077869,...,0.318953,-0.424366,1.490576,-0.041947,-1.370295,-0.601832,-0.848489,0.136065,-0.137326,1.329039
4,153259845_1_11_373_438_380_418.png,-4.282461,-2.235443,-2.527931,-1.657388,-0.944619,0.314474,-1.192246,2.033174,-2.018957,...,0.038974,1.476367,3.578484,0.167474,-1.553083,2.114680,-2.547345,-2.314631,-1.100696,9.076136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6686,112615698_0_25_1163_183_272_291.png,-0.475497,-2.973395,0.462854,5.043090,-1.432538,-2.691566,-1.190921,4.468680,-1.522096,...,1.228024,2.532765,2.962596,1.614798,-0.090937,-2.147948,-0.854707,1.041657,1.014960,-0.124583
6687,135351916_1_11_0_0_207_89.png,-1.318421,0.007685,0.116634,0.165788,-0.773530,0.123460,-1.098608,0.391126,-0.847334,...,0.003732,-0.148203,0.900114,-0.170786,-0.544440,-0.940026,-1.760076,-0.959367,-0.714172,3.435309
6688,122909668_1_15_0_0_252_64.png,-0.492056,0.087083,-0.079785,0.300682,-0.065463,-0.315982,-0.908591,0.006134,-0.431358,...,-0.057557,-0.149758,-0.523733,-0.077860,-0.304312,-0.218601,-1.470632,-1.102011,-0.616816,2.659305
6689,122716565_1_15_0_0_234_67.png,-0.573023,0.167290,-0.125009,0.293578,-0.493625,-0.350255,-0.916397,0.293158,-0.415224,...,-0.192334,-0.005288,-0.412890,-0.144492,-0.287466,-0.341712,-1.675318,-1.049135,-0.644267,2.740156


In [28]:
# 4. logit을 softmax 확률로 변환 후, 원본 이미지명 별로 확률, x, y, w, h 저장한 dict 생성
import numpy as np

def logit2conf(logit):
    logit = np.array(logit)
    exp_logit = np.exp(logit)
    plogit = exp_logit / np.sum(exp_logit)

    return plogit
    # indices = np.argsort(plogit)[-3:][::-1]
    # confs = plogit[indices]
    # result = []
    # for class_id, conf in zip(indices, confs):
    #     result.append([class_id, conf])

    # return result

cnn_dict = defaultdict(list)
for i in range(len(cnn_pred)):
    crop_name = cnn_pred.iloc[i]['crop_name']

    file_name, _, _, x, y, w, h = crop_name.split('_')
    file_name = file_name + '.png'

    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h.replace('.png', ''))
    bbox = [x, y, w, h]
    logit = []
    for cls_id in range(35):
        logit.append(cnn_pred.iloc[i][str(cls_id)])
    plogit = logit2conf(logit)
    cnn_dict[file_name].append([bbox, plogit])

In [29]:
# 5. yoloy 예측을 원본 이미지명 별로 클래스, 확률, x, y, w, h 저장한 dict 생성
yolo_dict = defaultdict(list)
for i in range(len(yolo_pred)):
    file_name = yolo_pred.iloc[i]['file_name']
    class_id = int(yolo_pred.iloc[i]['class_id'])
    conf = float(yolo_pred.iloc[i]['confidence'])
    x = int(yolo_pred.iloc[i]['point1_x'])
    y = int(yolo_pred.iloc[i]['point1_y'])
    w = int(yolo_pred.iloc[i]['point2_x'])
    h = int(yolo_pred.iloc[i]['point3_y'])
    yolo_dict[file_name].append([class_id, conf, x, y, w, h])

In [30]:
# 6. test 이미지명 전체를 돌면서 yolo dict의 이미지 별 Bbox들을 돌면서 분류 모델의 bbox들과 iou가 0.5 이상인 bbox의 확률을 분류 모델의 bbox 의 확률로 보정
# 보정 방법
    # 6-1. 패치별로 yolo 예측은 여러 개, 분류 모델 확률은 한 개 존재
    # 6-2. yoloy bbox들의 conf를 클래스 위치에 맞게 분류 모델 확률의 자리에 더하기
    # 6-2. 더해진 확률을 softmax로 변환
    # 6-3. softmax 확률중 top 3의 클래스와 확률을 저장
    # 6-4. top 3의 클래스와 확률을 가장 conf가 높았던 yolo의 Bbox 좌표에 저장(즉, 총 3개의 bbox 저장됨)

def iou(bbox1, bbox2):
    _, _, xmin1, ymin1, _, _, xmax1, ymax1, _, _ = bbox1
    _, _, xmin2, ymin2, _, _, xmax2, ymax2, _, _ = bbox2

    # intersection
    xmin = max(xmin1, xmin2)
    ymin = max(ymin1, ymin2)
    xmax = min(xmax1, xmax2)
    ymax = min(ymax1, ymax2)
    intersection = max(0, xmax - xmin) * max(0, ymax - ymin)

    # union
    area1 = (xmax1 - xmin1) * (ymax1 - ymin1)
    area2 = (xmax2 - xmin2) * (ymax2 - ymin2)
    union = area1 + area2 - intersection

    return intersection / union


weight = 10

final_dict = defaultdict(list)

# cnn의 이미지 별로
for file_name, infos in cnn_dict.items():
    # bbox 들을 하나씩 가져옴
    for i,info in enumerate(infos):
        cnn_bbox, logit = info
        cnn_x, cnn_y, cnn_w, cnn_h = cnn_bbox
        cnn_bbox = [0, 0, cnn_x, cnn_y, cnn_x+cnn_w, cnn_y, cnn_x+cnn_w, cnn_y+cnn_h, cnn_x, cnn_y+cnn_h] # 0, 0, 은 dummy class, dummy conf
        
        # cnn bbox 별로 yolo bbox 들을 하나씩 가져와 비교 후 softmax 확률에 conf로 보정
        for info in yolo_dict[file_name]:
            class_id, conf, x, y, w, h = info
            yolo_bbox = [class_id, conf, x, y, x+w, y, x+w, y+h, x, y+h]

            if iou(cnn_bbox, yolo_bbox) > 0.5:
                logit[class_id] += weight * conf
        
        # 더해진 확률들 0~1 사이로 변환
        # plogit = logit2conf(logit)
        plogit = logit / np.sum(logit)
        indices = np.argsort(plogit)[-3:][::-1]

        # 배경 클래스 처리
        background_prob = 0
        # background_prob = plogit[34] # 아직 시도 단계

        if 34 in indices:
            # 넘파이 배열의 34 원소 위치 찾기
            idx = np.where(indices == 34)[0][0]
            indices = indices[:idx] # 배경이 나오기 전까지만 저장
            background_prob = plogit[34] # 배경 확률값 저장

        # softmax 확률에서 배경 확률 빼기
        confs = plogit[indices]
        for i, conf in enumerate(confs):
            confs[i] = conf - background_prob

        # softmax 확률중 top 3의 클래스와 확률을 저장
        for index, conf in zip(indices, confs):
            if index == 34:
                continue
            conf = round(conf, 5)
            final_dict[file_name].append([index, conf, cnn_x, cnn_y, cnn_w, cnn_h])



In [31]:
# 7. 6.의 결과를 df로 변환 후 저장
df = {
    'file_name': [],
    'class_id': [],
    'confidence': [],
    'point1_x': [],
    'point1_y': [],
    'point2_x': [],
    'point2_y': [],
    'point3_x': [],
    'point3_y': [],
    'point4_x': [],
    'point4_y': []
}
for file_name, infos in final_dict.items():
    for class_id, confidence, x, y, w, h in infos:
        if class_id == 34 or confidence <= 0.00001:
            continue
        df['file_name'].append(file_name)
        df['class_id'].append(class_id)
        df['confidence'].append(confidence)
        df['point1_x'].append(x)
        df['point1_y'].append(y)
        df['point2_x'].append(x + w)
        df['point2_y'].append(y)
        df['point3_x'].append(x + w)
        df['point3_y'].append(y + h)
        df['point4_x'].append(x)
        df['point4_y'].append(y + h)

df = pd.DataFrame(df).sort_values(by=['file_name', 'confidence'], ascending=[True, False])
df
        

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
11232,064442001.png,11,0.33108,0,0,113,0,113,113,0,113
11233,064442001.png,24,0.22249,0,0,113,0,113,113,0,113
11235,064442001.png,2,0.21296,1138,181,1429,181,1429,497,1138,497
11236,064442001.png,24,0.20204,1138,181,1429,181,1429,497,1138,497
11237,064442001.png,16,0.13055,1138,181,1429,181,1429,497,1138,497
...,...,...,...,...,...,...,...,...,...,...,...
7795,183928410.png,33,0.00087,934,0,1192,0,1192,108,934,108
7797,183928410.png,2,0.00009,684,49,919,49,919,308,684,308
9269,184121675.png,27,0.75104,878,556,1149,556,1149,859,878,859
9270,184121675.png,2,0.12584,878,556,1149,556,1149,859,878,859


In [32]:
df.to_csv(csv_dir.replace('.csv','_cnn.csv'), index=False)